In [ ]:
import pandas as pd
import numpy as np
import re

# 여가 관련 자료

## 일과 여가 생활 균형

In [ ]:
leisure = pd.read_csv("G:/내 드라이브/취미/일과_여가생활_간_균형_20230510094439.csv", encoding='cp949')
# leisure.info()
# leisure.head(10)

In [ ]:
leisure_T = leisure.T   # 행열 전환
# leisure_T.head(5)

In [ ]:
leisure_T.replace("-",0,inplace=True)

In [ ]:
leis_uq = leisure.iloc[:,0].unique()
leis_uq = leis_uq.tolist()   # ndarray -> list
leis_uq = [re.sub(r"\(+.\)","",uq.replace(" ","_")) for uq in leis_uq]  # 괄호와 괄호 안 글자, 공백 제거
print(leis_uq)

In [ ]:
leisure_T.columns = leisure_T.iloc[0,:]
# leisure_T.info()

In [ ]:
columns_nospace = [re.sub(r"\(+.\)", "", col.replace(" ","_")) for col in leisure_T.columns]

In [ ]:
leisure_T.columns = columns_nospace
print(leisure_T.columns)

In [ ]:
# 통계 분류별로 데이터 프레임 분류

for cat in leis_uq : 
    globals()["leisure_{}".format(cat)] = leisure_T[cat]
    
    if type(globals()["leisure_{}".format(cat)]) == pd.core.series.Series :   # type이 시리즈라면
        globals()["leisure_{}".format(cat)] = pd.DataFrame(globals()["leisure_{}".format(cat)])   # 시리즈를 데이터프레임으로 변경
        
    globals()["leisure_{}".format(cat)].columns = globals()["leisure_{}".format(cat)].iloc[1,:]
    globals()["leisure_{}".format(cat)] = globals()["leisure_{}".format(cat)].iloc[2:,:]
    globals()["leisure_{}".format(cat)].iloc[:,1:] = globals()["leisure_{}".format(cat)].iloc[:,1:].astype(float)

In [ ]:
leis_uq2 = leis_uq[1:]
print(leis_uq2)

In [ ]:
for uq2 in leis_uq2 : 
    globals()["leis_{0}_{1}".format(uq2, "합본")] = pd.concat([leisure_통계분류, globals()["leisure_{}".format(uq2)]],axis=1)

In [ ]:
## 연도 리스트
year_List = [2016,2018,2019,2020,2021]

In [ ]:
for uniq in leis_uq2 : 
    idx_List = globals()["leis_{0}_{1}".format(uniq, "합본")].index.to_list()   # 분리된 데이터프레임 인덱스 리스트로 반환
    
    for yr in year_List : 
        sep_year = []    # 정규표현식에 만족하는 인덱스를 저장할 리스트
        
        for idx in idx_List :
            find_yr = re.findall(str(yr)+"[.0-9]*",idx)     # 연도 + '.' or 숫자 1개 이상 포함돼 있는 인덱스 저장 

            if find_yr:      # 빈 리스트가 아니라면 (findall은 조건에 안 맞으면 빈 리스트 반환)
                sep_year.append(find_yr[0])

        Last_idx = len(sep_year)
        globals()["leis_{0}_{1}".format(uniq, str(yr))] = globals()["leis_{0}_{1}".format(uniq, "합본")][:][sep_year[0]:sep_year[Last_idx-1]]    # 행 인덱스로 슬라이싱, 행 인덱스로 슬라이싱할 때 콜론 뒤 값을 포함해서 슬라이싱한다.
        pattern = str(yr) + r'[.0-9]*'
        new_index = globals()["leis_{0}_{1}".format(uniq, str(yr))].index.map(lambda x: re.sub(pattern, str(yr), x))   # map 함수를 이용해 인덱스마다 정규 표현식 sub를 적용
        globals()["leis_{0}_{1}".format(uniq, str(yr))].index = new_index
        globals()["leis_{0}_{1}".format(uniq, str(yr))].index = globals()["leis_{0}_{1}".format(uniq, str(yr))].index.astype(int)   # 인덱스를 int 타입으로
        globals()["leis_{0}_{1}".format(uniq, str(yr))].rename(columns={"통계분류(2)" : "분류"}, inplace=True)   # 컬럼명 변경
        globals()["leis_{0}_{1}".format(uniq, str(yr))] = globals()["leis_{0}_{1}".format(uniq, str(yr))].reset_index()   # reset_index
        globals()["leis_{0}_{1}".format(uniq, str(yr))].rename(columns={"index" : "연도"}, inplace=True)    # 인덱스가 컬럼으로 바뀌고, 그 컬럼명을 변경
        globals()["leis_{0}_{1}".format(uniq, str(yr))] = globals()["leis_{0}_{1}".format(uniq, str(yr))].set_index(["연도","분류"])   # 멀티 인덱스 설정


# 전체 -> 리스트 중에서 2016년이면 리스트에 저장 -> 리스트에 저장된 첫번째와 마지막 인덱스로 슬라이싱 -> 새로운 데이터 프레임 생성

In [ ]:
leis_17개_시도별_2016_T = leis_17개_시도별_2016.T
leis_17개_시도별_2016_T.head(10)

In [ ]:
df = leis_17개_시도별_2016_T.stack(level=0)
pd.DataFrame(df)


In [ ]:
pd.concat([leis_17개_시도별_2016, leis_가구소득별_2016],axis=1)


## 일과 여가 생활 균형 2

In [37]:
leisure2 = pd.read_csv("G:/내 드라이브/취미/일과_여가생활_간_균형_20230510094439.csv", encoding='cp949')

In [83]:
year_List2 = list(range(2016,2022))
year_List2 = [value for value in year_List2 if value != 2017]   # 2017년도 빼기

leis_sep = leisure2.iloc[:,:2]

for yr in year_List2 : 
    pattern = str(yr)
    globals()["leis2_{0}".format(str(yr))] = leisure2.filter(like=pattern)    # 조건에 만족하는 데이터 프레임 반환
    globals()["leis2_{0}".format(str(yr))] = pd.concat([leis_sep, globals()["leis2_{0}".format(str(yr))]], axis=1)     # 왼쪽 오른쪽으로 합치기, 인덱스가 다르면 outer join
    globals()["leis2_{0}".format(str(yr))] = globals()["leis2_{0}".format(str(yr))].replace("-",0)
    globals()["leis2_{0}".format(str(yr))].columns = globals()["leis2_{0}".format(str(yr))].iloc[0,:].to_list()
    globals()["leis2_{0}".format(str(yr))] = globals()["leis2_{0}".format(str(yr))].iloc[1:,:]
    globals()["leis2_{0}".format(str(yr))].iloc[:,2:] = globals()["leis2_{0}".format(str(yr))].iloc[:,2:].round(0).astype(int)
    globals()["leis2_{0}".format(str(yr))] = globals()["leis2_{0}".format(str(yr))].set_index(["통계분류(1)","통계분류(2)"])
    # globals()["leis2_{0}".format(str(yr))] = globals()["leis2_{0}".format(str(yr))].apply(lambda x : round(x))
    globals()["leis2_{0}".format(str(yr))].iloc[:,1:] = globals()["leis2_{0}".format(str(yr))].iloc[:,1:].apply(lambda x : x * leis2_2016.iloc[:,0] / 100)  # 백분율로 표시된 값을 실제 값으로 변환
    globals()["leis2_{0}".format(str(yr))] = globals()["leis2_{0}".format(str(yr))].iloc[:,1:]  # 합계 부분 삭제

ValueError: invalid literal for int() with base 10: '1.6'

In [84]:
leis2_2016.round(0)

,통계분류(1),통계분류(2),표본수,일에 더 집중하고있다,일에 조금 더 집중하고 있다,다소 일에 집중하고 있다,균형을 이루고 있다,다소 여가에 더 집중하고있다,여가에 조금 더 집중하고 있다,여가에 더 집중하고 있다
1,전체,소계,10602,1.6,8.6,20.4,35.5,21.8,11.4,0.7
2,성별,남성,5179,1.7,9,22.8,34.4,20.5,10.9,0.6
3,성별,여성,5423,1.5,8.1,18.1,36.5,23,11.9,0.9
4,연령별,15~19세,827,1.9,11.6,26.4,34.4,19.3,5.9,0.5
5,연령별,20대,1476,0.8,7.9,20.5,39.5,21.7,9.2,0.4
...,...,...,...,...,...,...,...,...,...,...
63,17개 시도별,경남,719,1.4,5.1,14.8,37.1,29.2,12,0.3
64,17개 시도별,제주,300,10.8,20.9,18.8,29.6,6.7,10.7,2.7
65,장애여부별,장애등록,0,0,0,0,0,0,0,0
66,장애여부별,미등록,0,0,0,0,0,0,0,0


In [ ]:
leis2_2016.set_index(["통계분류(1)","통계분류(2)"]).unstack()

In [ ]:
leis2_2016.columns = leis2_2016.iloc[0,:].to_list()